# Neurocombat TOPMRI and HELIUS harmonized datasets

Note this must be run in the `neuron` or `neuron_go` environment

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

from neuroCombat import neuroCombat

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
# Datasets for this work
EDIS_path = '../our_datasets/EDIS/'
HELIUS_path = '../our_datasets/HELIUS/'
Insight46_path = '../our_datasets/Insight46/'
HELIUS_path = '../our_datasets/HELIUS/'
MRI_path = '../our_datasets/StrokeMRI/'
TOP_path = '../our_datasets/TOP/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
EDIS_file = os.path.join(EDIS_path, file_name)

Insight46_file = os.path.join(Insight46_path, file_name)
HELIUS_file = os.path.join(HELIUS_path, file_name)


#HELIUS = pd.read_csv(HELIUS_file)
TOP = pd.read_csv(TOP_file, index_col=0 )
MRI = pd.read_csv(MRI_file, index_col=0)
HELIUS = pd.read_csv(HELIUS_file, index_col=0) # 

In [ ]:
HELIUS.Sex.unique()

In [ ]:

HELIUS.head(3)

In [ ]:
TOP = TOP.drop(['Site', 'ID'], axis= 1)
TOP.head(3)

In [ ]:
MRI = MRI.drop(['Site', 'ID'],axis = 1)
MRI.tail(3)

In [ ]:
TOPMRI = pd.concat([MRI ,TOP])
TOPMRI.head(3)

In [ ]:
HELIUS = HELIUS.drop(['Site', 'ID'],axis = 1)
HELIUS.tail(3)

In [ ]:
TOPMRI.shape

In [ ]:
both_togetherF, ftF, btF, feature_dictF, len1, len2 = har.prep_for_neurocombat(TOPMRI, HELIUS)

In [ ]:
both_togetherF

In [ ]:
# our_mri_data = pd.read_csv(filename_mri)
# our_top_data = pd.read_csv(filename_top)

In [ ]:
# # save off csv
# both_together.to_csv('both_top_mri_together.csv')

In [ ]:
# # make and save of csv of features only
# features_only = both_together[2:]
# #features_only.to_csv('features_only_top_mri.csv')

In [ ]:
ftF.to_csv('ftF_top_mri_HELIUS.csv')

In [ ]:
data = np.genfromtxt('ftF_top_mri_HELIUS.csv', delimiter=",", skip_header=1)
data = data[:, 1:]
data

In [ ]:
TOPMRI.head(3)

In [ ]:
both_togetherF.head(3)

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len1
last_columns_as_two = [2] * len2
covars = {'batch':first_columns_as_one + last_columns_as_two,
          #'Site': both_togetherF.loc['Site',:].values.tolist(), 
          'Sex':both_togetherF.loc['Sex',:].values.tolist(),
           'Age':both_togetherF.loc['Age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
# specifify sex as categorical
categorical_cols = ['Sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'batch'
our_continuous_col=['Age']
#Harmonization step:
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col_mine,
    continuous_cols=our_continuous_col,
    categorical_cols=categorical_cols)["data"]

# (n_dat, covars, batch_col, continuous_cols=continuous_cols,
#                                     categorical_cols=categorical_cols)

In [ ]:
neurocombat = pd.DataFrame(data_combat)
neurocombat

In [ ]:
topperF = har.make_topper(btF,'Age', 'Sex')

In [ ]:
bottom = neurocombat.reset_index(drop=False)
bottom = bottom.rename(columns={"index": "char"})
bottom.columns = topperF.columns

In [ ]:
back_together = pd.concat([topperF, bottom])
back_together = back_together.T
#back_together

In [ ]:
new_header = back_together.iloc[0] #grab the first row for the header
back_together.columns = new_header #set the header row as the df header
back_together = back_together[1:]
#back_together

In [ ]:
back_together.head(3)

In [ ]:
neuro_harm_HELIUS =back_together.tail(len2)
neuro_harm_topmri =back_together.head(len1)

In [ ]:
neuro_harm_topmri = neuro_harm_topmri.rename(feature_dictF, axis='columns')
neuro_harm_HELIUS = neuro_harm_HELIUS.rename(feature_dictF, axis='columns')

In [ ]:
neuro_harm_HELIUS = neuro_harm_HELIUS.reset_index()
neuro_harm_HELIUS = neuro_harm_HELIUS.rename(columns={"index": "participant_id"})
neuro_harm_HELIUS

Save off to harmonized csv files

In [ ]:
neuro_harm_topmri = neuro_harm_topmri.reset_index()
neuro_harm_topmri = neuro_harm_topmri.rename(columns={"index": "participant_id"})
neuro_harm_topmri

In [ ]:
neuro_harm_HELIUS.to_csv('harm_results/neuro_harm_HELIUS.csv')
neuro_harm_topmri.to_csv('harm_results/neuro_harm_topmri_from_HELIUS.csv')

In [ ]:
#neuro_harm_HELIUS.columns

In [ ]:
number_columns = ['Age', 'Sex', 'GM_vol', 'WM_vol', 'CSF_vol',
       'GM_ICVRatio', 'GMWM_ICVRatio', 'WMHvol_WMvol', 'WMH_count',
       'ACA_B_CoV', 'MCA_B_CoV', 'PCA_B_CoV', 'TotalGM_B_CoV',
       'ACA_B_CBF', 'MCA_B_CBF', 'PCA_B_CBF', 'TotalGM_B_CBF',]

# Data quality check

In [ ]:
TOPMRI

In [ ]:
neuro_harm_topmri

In [ ]:
neuro_harm_HELIUS.isna().sum().sum()

In [ ]:
neuro_harm_topmri.isna().sum().sum()

In [ ]:
#neuro_harm_top.head(3)
(neuro_harm_HELIUS[number_columns] < 0).sum()

In [ ]:
(neuro_harm_topmri[number_columns] < 0).sum()